In [1]:
import time
import cv2 as cv
import numpy as np
import pyautogui
import torch
import os
import yaml
from mss.linux import MSS as mss
from ultralytics import YOLO
from datasets import load_dataset
from pathlib import Path

c:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# loading the dataset from huggingface that is in COCO (Common Objects in Context) format
ds = load_dataset("keremberke/csgo-object-detection", name="full")

# start training from scratch
model = YOLO("yolov8m.yaml").to("cuda")

In [7]:
def make_folders(root, img_lab, train_val_test):
    needed_path = root / img_lab / train_val_test
    needed_path.mkdir(parents = True, exist_ok=True)
    
# writing a function that extracts the labels for each image in the dataset in YOLO format
def save_img_label(df, train_val_test, output_dir, rounding = 6):
    
    output_dir_image = output_dir / "images"
    output_dir_label = output_dir / "labels"
    
    # ensure the output directory exists
    os.makedirs(output_dir_image, exist_ok = True)
    os.makedirs(output_dir_label, exist_ok = True)
    
    for index, item in enumerate(df[train_val_test]):
        current_obj = item['objects']
        label = ""
        for i, _ in enumerate(current_obj['category']):
            name = current_obj['category'][i]
            # the width and height of each picture is known to be 416 x 416, we need this to normalize everything
            bbox = [np.round(x/416, rounding) for x in current_obj['bbox'][i]]
            # bbox is in (xmin, ymin, width, height) format, we need to convert it to (xcenter, ycenter, width, height) format
            xcenter = np.round(bbox[0] + bbox[2]/2, rounding)
            ycenter = np.round(bbox[1] + bbox[3]/2, rounding)
            # now we create the label
            label += f"{name} {xcenter} {ycenter} {bbox[2]} {bbox[3]} \n"
            
        img = item['image']

        output_path_label = os.path.join(output_dir_label / train_val_test, f"{index}.txt")
        output_path_image = os.path.join(output_dir_image / train_val_test, f"{index}.JPG")
        
        
        with open(output_path_label, 'w') as f:
            f.write(label)
            
        img.save(output_path_image)   

In [9]:
root_path = Path("yolo_data")

for i in ["images", "labels"]:
    for j in ["train", "validation", "test"]:
        make_folders(root_path, i, j)
        
for i in ["train", "validation", "test"]:
    save_img_label(ds, i, root_path)

In [10]:
model = model.train(data = "config.yaml", epochs = 15)

New https://pypi.org/project/ultralytics/8.2.75 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8s.yaml, data=config.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

train: Scanning C:\Users\yangs\Documents\Python Projects\YOLO-Head-Detection\yolo_data\labels\train... 3879 images, 573 backgrounds, 0 corrupt: 100%|██████████| 3879/3879 [00:03<00:00, 1263.23it/s]


train: New cache created: C:\Users\yangs\Documents\Python Projects\YOLO-Head-Detection\yolo_data\labels\train.cache


val: Scanning C:\Users\yangs\Documents\Python Projects\YOLO-Head-Detection\yolo_data\labels\validation... 383 images, 57 backgrounds, 0 corrupt: 100%|██████████| 383/383 [00:00<00:00, 476.28it/s]


val: New cache created: C:\Users\yangs\Documents\Python Projects\YOLO-Head-Detection\yolo_data\labels\validation.cache
Plotting labels to runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train4
Starting training for 15 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      15/15       3.8G      1.332     0.8744      1.474         16        640: 100%|██████████| 243/243 [00:41<00:00,  5.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:02<00:00,  4.98it/s]

                   all        383        831      0.866      0.764       0.85      0.477



15 epochs completed in 0.223 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 22.5MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics YOLOv8.2.74  Python-3.12.5 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
YOLOv8s summary (fused): 168 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:02<00:00,  4.48it/s]


                   all        383        831      0.866      0.765      0.849      0.477
                    ct        189        244      0.867      0.857      0.902      0.537
                cthead        185        237      0.817       0.64       0.76      0.341
                     t        137        183      0.891      0.836      0.911      0.614
                 thead        123        167       0.89      0.726      0.825      0.415
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train4
